# Model Evaluation

##### *In which we hope all the work we put into our model pays off.*

###### [GitHub Repository](https://github.com/ahester57/ai_workshop/tree/master/notebooks/time_for_crab/3-evaluation)

###### [Notebook Viewer](https://nbviewer.jupyter.org/github/ahester57/ai_workshop/blob/master/notebooks/time_for_crab/3-evaluation/evaluation.ipynb)

###### [Kaggle Dataset](https://www.kaggle.com/sidhus/crab-age-prediction)


### Define Constants

In [2]:
CACHE_FILE = '../cache/designrcrabs.feather'
TEST_CACHE_FILE = '../cache/splitcrabs_test.feather'

NEXT_NOTEBOOK = '../Hester-CS5300-Time-for-Crab.ipynb'

PREDICTION_TARGET = 'Age'    # 'Age' is predicted
DATASET_COLUMNS = ['Sex_F','Sex_M','Sex_I','Length','Diameter','Height','Weight','Shucked Weight','Viscera Weight','Shell Weight',PREDICTION_TARGET]
REQUIRED_COLUMNS = [PREDICTION_TARGET]


### Import Libraries

In [3]:
from notebooks.time_for_crab.mlutils import data_downcasting, display_df

from sklearn.metrics import roc_curve, auc

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

pd.set_option('mode.copy_on_write', True)


### Load Data from Cache

In the [previous section](../2-features/features.ipynb), we saved the normalized data to the cache.


In [4]:
crabs = pd.read_feather(CACHE_FILE)
display_df(crabs, show_distinct=True)


DataFrame shape: (3114, 11)
First 5 rows:
        Length  Diameter    Height    Weight  Shucked Weight  Viscera Weight  \
1698  0.500977  0.394531 -0.725586 -0.199707       -0.126953       -0.445801   
1361  0.743164  0.713867 -0.645996  0.489258        0.507812       -0.045898   
1972  0.013672 -0.025391 -0.787598 -0.706543       -0.755859       -0.750977   
960   0.163086  0.126953 -0.813965 -0.537109       -0.616211       -0.527344   
2639  0.716797  0.748047 -0.690430  0.099609       -0.041504       -0.026367   

      Shell Weight  Sex_F  Sex_I  Sex_M  Age  
1698     -0.362305  False  False   True    8  
1361      0.100586  False  False   True   12  
1972     -0.701660   True  False  False    9  
960      -0.579102  False  False   True   11  
2639      0.159180   True  False  False   15  
<class 'pandas.core.frame.DataFrame'>
Index: 3114 entries, 1698 to 645
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 

## Regression Model Evaluation

### Compute the ROC Curve

This is a regression problem, so the ROC curve is more of a challenge.

First, we need to convert our predictions into True/False. Since we are trying to predict a crab's age,
a threshold of 2 years is a good starting point. If the prediction is within 2 years of the actual, we'll consider it a success.

#### Convert Predictions to True/False


In [ ]:
#